Run the following cell to install dependencies required to run this notebook

In [ ]:
!pip install -r https://raw.githubusercontent.com/meyresearch/intro_to_md/main/requirements.txt

In [ ]:
!pip install openmm nglview==2.7.7 ipywidgets==7.7.1

Run the following cell to clone the files required to run this notebookk

In [ ]:
import os

REPO_URL = "https://github.com/meyresearch/intro_to_md.git"
REPO_DIR = "intro_to_md"

if not os.path.isdir(REPO_DIR):
    print(f"Cloning {REPO_URL}…")
    !git clone --depth 1 $REPO_URL

%cd $REPO_DIR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

REPO = "https://github.com/meyresearch/intro_to_md.git"
DEST = "/content/drive/MyDrive/intro_to_md"

if not os.path.isdir(DEST):
    print(f"Cloning into Drive…")
    !git clone --depth 1 $REPO $DEST

%cd $DEST

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

# Simulating a protein in a box of water

Based on [Justin Lemkul's lysozyme in water tutorial](http://www.mdtutorials.com/gmx/lysozyme/index.html) and the [OpenMM tutorial](https://openmm.github.io/openmm-cookbook/latest/notebooks/tutorials/protein_in_water.html#Protein-in-Water).

# 1. Import libraries

In [ ]:
from openmm.app.pdbfile import PDBFile
from openmm.app import PME, HBonds, Simulation, pdbreporter, statedatareporter, dcdreporter
from openmm import LangevinMiddleIntegrator, Platform, MonteCarloBarostat, MinimizationReporter
from openmm.app.forcefield import ForceField
from openmm.app.modeller import Modeller
import openmm.unit as unit
import nglview as nv
import sys

# 2. Load in the PDB file

In [ ]:
pdb = PDBFile("input/kpc2.pdb")

# 3. Define the force field and water model

In [ ]:
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")

# 4. Solvate the protein

Since proteins are found inside the body, we simulate the enivronment in the body with a box of water. We also add neutralising Na and Cl ions, so that the total charge in the box of water is neutral. 

This command creates a box that has edges at least 1 nm away from the solute and fills it with water molecules. 

In [ ]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, padding=1.0*unit.nanometer)

# 5. Visualise the solvated system

First, save a temporary PDB file of the solvated system:

In [ ]:
pdbfile = PDBFile.writeFile(modeller.topology, modeller.positions, open("results/output/solvated.pdb", "w"))

In [ ]:
view = nv.show_file("results/output/solvated.pdb") 
view.add_representation("cartoon", colorScheme="residueindex")
view

In [ ]:
view.add_representation("ball+stick", selection="water")
view

# 6. Setup the simulation description 

We now need to combine our molecular topology and the forcefield to create a complete description of the system. This is done using the ForceField object’s `createSystem()` function. We then create the integrator, and combine the integrator and system to create the `Simulation` object. Finally, we set the initial atomic positions.

In [ ]:
# First, define the system 
system = forcefield.createSystem(modeller.topology, 
                                 nonbondedMethod=PME, 
                                 nonbondedCutoff=1.0*unit.nanometer, 
                                 constraints=HBonds)


In [ ]:
# Define the integrator, i.e. how the simulation is calculated at each step
integrator = LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds)


In [ ]:
# Define the simulation and set initial positions
platform = Platform.getPlatformByName("CUDA") # Allows us to run simulations faster
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

# 7. Minimise the potential energy of the system

We need to make sure we are at a local energy minimum before doing the simulation

In [ ]:
print("Minimising energy")
simulation.minimizeEnergy()

# 8. Write out minimised coordinates

In [ ]:
# After minimization, get the final positions of the system
positions = simulation.context.getState(getPositions=True).getPositions()

with open("results/output/minimised_structure.pdb", "w") as file:
    PDBFile.writeFile(simulation.topology, positions, file)

# 9. Relax to the correct temperature

First, we want to setup "reporters" that allow us to write the trajectory to a file, and then we can run the NVT relaxation simulation.

In [ ]:
simulation.reporters.append(dcdreporter.DCDReporter("results/output/nvt.dcd", # define trajectory file to write to 
                                                    reportInterval=100)) # write every 100 steps to file
simulation.reporters.append(statedatareporter.StateDataReporter(
    file="results/output/temperature.log",  # define log file to write temperature to
    reportInterval=100, # write every 100 steps to file
    step=True,
    temperature=True
))

In [ ]:
print("Running NVT")
simulation.step(10000) # run for 10000 * 0.004 picoseconds = 40 ps 

# 10. Relax to correct pressure 

We first need to add a barostat to be able to control the pressure (and the volume)

In [ ]:
system.addForce(MonteCarloBarostat(1*unit.bar, 300*unit.kelvin))
simulation.context.reinitialize(preserveState=True)


In [ ]:
simulation.reporters.append(dcdreporter.DCDReporter("results/output/npt.dcd", # define trajectory file to write to 
                                                    reportInterval=100)) # write every 100 steps to file
simulation.reporters.append(statedatareporter.StateDataReporter(
    file="results/output/density.log",  # define log file to write density to
    reportInterval=100, # write every 100 steps to file
    step=True,
    density=True, 
))

In [ ]:
print("Running NPT")
simulation.step(10000) # run for 10000 * 0.004 picoseconds = 40 ps 

# 11. Run production simulation

Now we have reached the correct pressure and temperature, we can run our final simulation. 

In [ ]:
simulation.reporters.clear() # Clear all old reporters, so that we don't overwrite them
simulation.reporters.append(dcdreporter.DCDReporter("results/output/md.dcd", # define trajectory file to write to 
                                                    reportInterval=1000)) # write every 1000 steps to file
simulation.reporters.append(statedatareporter.StateDataReporter(
    file="results/output/md.log",  # define log file to write volume to
    reportInterval=1000, # write every 1000 steps to file
    step=True,
    density=True, 
    temperature=True,
    potentialEnergy=True,
))

In [ ]:
print("Running MD")
simulation.step(25000)